In [1]:
import importlib
import data_utils
importlib.reload(data_utils)
from data_utils import PhonemeDataset
from mlp_mixer import MLPMixer

import numpy as np
import torch 
from torch.utils.data import DataLoader
import torchvision.transforms.v2 as transforms

import pytorch_lightning as pl
from lightning import Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor

torch.autograd.set_detect_anomaly(True)

/home/abunnell/.conda/envs/torch_scipy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load data

In [2]:
transform = None
batch_size = 32

train_loader = DataLoader(
    PhonemeDataset(
        data_filename='Data/Phoneme/train_X.npy',
        label_filename='Data/Phoneme/train_y.npy',
        transform=transform
    ), 
    batch_size=batch_size, 
    shuffle=True
)

val_loader = DataLoader(
    PhonemeDataset(
        data_filename='Data/Phoneme/valid_X.npy',
        label_filename='Data/Phoneme/valid_y.npy',
        transform=None
    ), 
    batch_size=batch_size, 
    shuffle=False
)

test_loader = DataLoader(
    PhonemeDataset(
        data_filename='Data/Phoneme/test_X.npy',
        label_filename='Data/Phoneme/test_y.npy',
        transform=None
    ), 
    batch_size=batch_size, 
    shuffle=False
)

# Check a batch of train data
for X, y in train_loader:
    print(f"Shape of X (batch, channels, timesteps): {X.shape}, shape of labels: {len(y)}")
    break

Shape of X (batch, channels, timesteps): torch.Size([32, 11, 220]), shape of labels: 32


# Create mlp-mixer model

In [3]:
# patch_class options are: "sequential1d", "random1d", "cyclical1d"
# patch_class options are: "sequential1d", "random1d", "cyclical1d"
patch_class = "sequential1d"

mixer = MLPMixer(num_classes=39, num_blocks=5, patch_size=10, hidden_dim=128, 
                 patch_class='cyclical1d', tokens_mlp_dim=64, channels_mlp_dim=32, padded_length=220)

x = torch.randn(32, 11, 220)

v = mixer(x)

In [4]:
mixer_trainer = Trainer()
mixer_trainer.fit(model=mixer, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/abunnell/.conda/envs/torch_scipy/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/home/abunnell/.conda/envs/torch_scipy/lib/python3.12/site-packages/lightning/pytorch/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
/home/abunnell/.conda/envs/torch_scipy/lib/python3.12/site-packages/lightning/py

Epoch 6:  11%|█         | 11/104 [00:00<00:02, 46.07it/s, v_num=17, train_loss_step=2.930, train_loss_epoch=3.230, train_acc=0.102, collapse_flg_train=1.25e+3]  

/home/abunnell/.conda/envs/torch_scipy/lib/python3.12/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
